# contextlib

Преимущества использования contextlib
* Упрощение кода: создание менеджеров контекста без необходимости писать отдельные классы.
* Лаконичность: меньше кода, так как yield разделяет логику входа и выхода из контекста.
* Читаемость: функции, созданные с @contextmanager, могут выглядеть более понятными и логичными.

contextlib особенно удобен для создания простых и одноразовых менеджеров контекста, которые требуют минимального кода и могут быть применены для часто встречающихся шаблонов, таких как временная смена директории, подавление ошибок или перенаправление вывода.

In [ ]:
import os
from contextlib import contextmanager

@contextmanager
def change_directory(path):
    original_path = os.getcwd()  # Сохраняем текущую директорию
    os.chdir(path)               # Переходим в новую директорию
    try:
        yield                    # Останавливаемся на этом месте
    finally:
        os.chdir(original_path)  # Возвращаемся в исходную директорию при выходе из контекста

# Пример использования
print("Текущая директория:", os.getcwd())
with change_directory("/tmp"):
    print("Временная директория:", os.getcwd())
print("Вернулись в директорию:", os.getcwd())

Текущая директория: /Users/tryadovoi/Desktop/itmo/yadro_school/lesson_06/practice
Временная директория: /private/tmp
Вернулись в директорию: /Users/tryadovoi/Desktop/itmo/yadro_school/lesson_06/practice


Пояснение кода:
* До yield: выполняется логика входа в контекст (смена директории).
* После yield: выполняется логика выхода из контекста (возврат в исходную директорию).
* try-finally блок: гарантирует, что мы вернёмся в исходную директорию даже при возникновении ошибки.

## contextlib.suppress

In [2]:
from contextlib import suppress

with suppress(FileNotFoundError):
    os.remove("non_existent_file.txt")

In [3]:
os.remove("non_existent_file.txt")

FileNotFoundError: [Errno 2] No such file or directory: 'non_existent_file.txt'

## contextlib.redirect_stdout

In [58]:
from contextlib import redirect_stdout

with open("output.txt", "w") as f:
    with redirect_stdout(f):
        print("Этот текст будет записан в файл output.txt")

## timer

In [69]:
import time
from contextlib import contextmanager

@contextmanager
def timer():
    start = time.time()
    yield
    end = time.time()
    print(f"Время выполнения: {end - start:.3f} секунд")

# Пример использования
with timer():
    sum(range(1000000)) 

Время выполнения: 0.019 секунд


## Менеджер контекста для транзакционного выполнения с откатом

In [70]:
from contextlib import contextmanager

class TransactionManager:
    def __init__(self):
        self.changes = []

    def add_change(self, change):
        self.changes.append(change)
    
    def commit(self):
        print("Изменения сохранены:", self.changes)
    
    def rollback(self):
        print("Откат изменений:", self.changes)
        self.changes.clear()

@contextmanager
def transaction(manager):
    try:
        yield manager
        manager.commit()
    except Exception:
        manager.rollback()
        raise

# Пример использования
manager = TransactionManager()

try:
    with transaction(manager) as txn:
        txn.add_change("Изменение 1")
        txn.add_change("Изменение 2")
        raise ValueError("Ошибка")  # Имитируем ошибку
except Exception as e:
    print("Произошла ошибка:", e)

Откат изменений: ['Изменение 1', 'Изменение 2']
Произошла ошибка: Ошибка


## Менеджер контекста для временного изменения переменных окружения

In [71]:
import os
from contextlib import contextmanager

@contextmanager
def temporary_env(**kwargs):
    original_env = {key: os.environ.get(key) for key in kwargs}
    os.environ.update(kwargs)
    try:
        yield
    finally:
        # Восстанавливаем первоначальные значения переменных окружения
        for key, value in original_env.items():
            if value is None:
                del os.environ[key]
            else:
                os.environ[key] = value

# Пример использования
print("Перед:", os.getenv("MY_VAR"))

with temporary_env(MY_VAR="Temporary Value"):
    print("Внутри:", os.getenv("MY_VAR"))

print("После:", os.getenv("MY_VAR"))

Перед: None
Внутри: Temporary Value
После: None


## Менеджер контекста для работы с множественными файлами (пакетное открытие и закрытие)

In [23]:
from contextlib import contextmanager

@contextmanager
def multi_file_open(*filenames, mode="r"):
    files = []
    try:
        # Открываем каждый файл и добавляем его в список
        for filename in filenames:
            files.append(open(filename, mode))
        yield files
    finally:
        # Закрываем все файлы при выходе из контекста
        for file in files:
            file.close()

# Пример использования
file_names = ["file1.txt", "file2.txt", "file3.txt"]

with multi_file_open(*file_names, mode="w") as files:
    for i, file in enumerate(files, 1):
        file.write(f"Запись в файл {i}\n")

## Менеджер контекста для динамического изменения прав доступа к файлу

In [72]:
import os
from contextlib import contextmanager

@contextmanager
def temporary_file_permissions(filepath, permissions):
    # Сохраняем текущие права доступа
    original_permissions = os.stat(filepath).st_mode
    os.chmod(filepath, permissions)  # Устанавливаем новые права доступа
    
    try:
        yield
    finally:
        # Восстанавливаем исходные права доступа
        os.chmod(filepath, original_permissions)

# Пример использования
with open("example.txt", "w") as f:
    f.write("Пример текста")

print("Исходные права доступа:", oct(os.stat("example.txt").st_mode))

# Временно устанавливаем доступ только для чтения
with temporary_file_permissions("example.txt", 0o444):
    print("Временные права доступа:", oct(os.stat("example.txt").st_mode))
    # Пробуем записать - ожидается ошибка

print("Восстановленные права доступа:", oct(os.stat("example.txt").st_mode))

Исходные права доступа: 0o100644
Временные права доступа: 0o100444
Восстановленные права доступа: 0o100644


## Менеджер контекста для отслеживания использования памяти 

In [79]:
import os
import psutil
from contextlib import contextmanager

@contextmanager
def memory_monitor():
    process = psutil.Process(os.getpid())
    start_memory = process.memory_info().rss / (1024 ** 2)  # Память в MB
    
    try:
        yield
    finally:
        end_memory = process.memory_info().rss / (1024 ** 2)  # Память после выполнения
        print(f"Память использована: {end_memory - start_memory:.2f} MB")

# Пример использования
with memory_monitor():
    big_list = [i for i in range(10**6)]  # Создаём крупный объект для тестирования памяти

Память использована: 2.00 MB
